# 参考

参考：类属性的实现 https://stackoverflow.com/questions/128573/using-property-on-classmethods  

Python操作符重载  
https://blog.csdn.net/geek_xiong/article/details/82598228  

# 定义规则对象接口

# 全局变量

- gText：表示待解析的原始字符串。  
- gTextLen：待解析字符串的长度，这是一个给定。  

# 函数

## 匹配函数

- 函数原型： (int[] newIndexs) IMatcher(int curIndex)  
- 返回值：是一个列表，表示匹配成功的新的游标列表。如果没有匹配成功，则列表为空。    
- 参数：curIndex，表示从哪个游标开始解析。  

## AndP

- 函数原型： IMatcher AndP(IMatcher[] matchers)  
- 返回值：匹配函数    
- 参数：2个或者多个匹配函数。    

In [ ]:
def AndP(*matchers):
    def matcher(curIndex):
        matchersNum=len(matchers)
        if(matchersNum<=0):raise Exception('20190814_1132:matchersNum<=0')
        #如果只有一个，直接返回匹配结果即可
        if(matchersNum==1):
            return (matchers[0])(curIndex)
        #第一个匹配项
        firstMatcher=matchers[0]
        #删除第一项，得到剩余的匹配项
        remainMatchers=matchers[1:]
        #第一个匹配项的匹配结果
        results=firstMatcher(curIndex)
        #如果匹配失败，则整个失败
        if(len(results)==0): return []
        #匹配成功的情况
        res=[]
        for newIndex in results:
            tmpResult=AndP(*remainMatchers)(newIndex)
            res.extend(tmpResult)
        #匹配结果合并去重
        res=list(set(res))
        #返回最终的结果
        return res
    return matcher

## OrP

- 函数原型： IMatcher OrP(IMatcher[] matchers)  
- 返回值：匹配函数    
- 参数：2个或者多个匹配函数。    

In [ ]:
def OrP(*matchers):
    def matcher(curIndex):
        res=[]
        for matcher in matchers:
            tmpResult=matcher(curIndex)
            res.extend(tmpResult)
        #匹配结果合并去重
        res=list(set(res))
        #返回最终的结果
        return res
    return matcher

## 匹配单字符

- 函数原型： IMatcher Char(char ch)  
- 返回值：匹配函数    
- 参数：ch，匹配的字符。    

In [ ]:
def Char(ch):
    def matcher(curIndex):
        if curIndex>=gTextLen: return []
        if curIndex<0: raise Exception('20190814_0951: curIndex<0')
        if gText[curIndex]==ch: 
            newIndex=curIndex+1
            return [newIndex]
        return []
    return matcher

# 主程序

In [ ]:
a=Char('a')
b=Char('b')
def S(curIndex):
    tmp=AndP(a,S)
    res=OrP(tmp,b)
    return res(curIndex)

In [ ]:
gText='ab'
gTextLen=len(gText)

def Run():
    res=S(0)
    print(f'res:{res}')
    for i in res:
        print(f'text:{gText[0:i]}')